In [96]:
# packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt

In [84]:
adata = sc.read_h5ad("C:/Users/jessi/OneDrive/Desktop/D17_FS3_processed.h5ad")

In [85]:
adata.obs

,sample,time,mouse,slide,rank,x,y,nCount_RNA,nFeature_RNA,percent.mt,...,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mito,log1p_total_counts_mito,pct_counts_mito,clusters
1040_137,17D-M2-1_FS3,D17,M2,FS3,6,0,137,2587,824,0.695787,...,2587.0,7.858641,40.896792,50.753769,64.514882,87.475841,18.0,2.944439,0.695787,2
1040_138,17D-M2-1_FS3,D17,M2,FS3,7,0,138,6453,1645,0.635363,...,6453.0,8.772455,36.246707,45.095305,55.726019,74.012087,41.0,3.737670,0.635363,1
1040_139,17D-M2-1_FS3,D17,M2,FS3,6,0,139,5708,1468,0.823406,...,5708.0,8.649799,37.193413,46.303434,57.620883,76.822004,47.0,3.871201,0.823406,1
1040_140,17D-M2-1_FS3,D17,M2,FS3,5,0,140,6226,1586,0.594282,...,6226.0,8.736650,35.448121,44.763893,56.424671,75.072278,37.0,3.637586,0.594282,1
1040_141,17D-M2-1_FS3,D17,M2,FS3,5,0,141,5500,1422,0.690909,...,5500.0,8.612685,35.345455,44.400000,56.072727,76.472727,38.0,3.663562,0.690909,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168_95,17D-M2-1_FS3,D17,M2,FS3,7,128,95,10858,2329,0.534168,...,10858.0,9.292749,33.799963,42.696629,53.158961,69.497145,58.0,4.077538,0.534168,5
1168_96,17D-M2-1_FS3,D17,M2,FS3,8,128,96,10857,2387,0.571060,...,10857.0,9.292658,32.614903,41.180805,51.690154,68.085106,62.0,4.143135,0.571060,5
1168_97,17D-M2-1_FS3,D17,M2,FS3,9,128,97,12143,2467,0.584699,...,12143.0,9.404591,34.291361,43.243021,54.097011,70.073293,71.0,4.276666,0.584699,2
1168_98,17D-M2-1_FS3,D17,M2,FS3,9,128,98,10388,2434,0.587216,...,10388.0,9.248503,33.423181,41.750096,51.540239,66.894494,61.0,4.127134,0.587216,6


In [86]:
# Convert the last column to labels
labels = adata.obs.iloc[:, -1].values
labels

['2', '1', '1', '1', '1', ..., '5', '5', '2', '6', '6']
Length: 27915
Categories (8, object): ['0', '1', '2', '3', '4', '5', '6', '7']

In [87]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y

# Remove the last column (assuming it contains cluster IDs)
adata.obs = adata.obs.drop(adata.obs.columns[-1], axis=1)
adata.obs

,sample,time,mouse,slide,rank,x,y,nCount_RNA,nFeature_RNA,percent.mt,...,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mito,log1p_total_counts_mito,pct_counts_mito
1040_137,17D-M2-1_FS3,D17,M2,FS3,6,0,137,2587,824,0.695787,...,6.715383,2587.0,7.858641,40.896792,50.753769,64.514882,87.475841,18.0,2.944439,0.695787
1040_138,17D-M2-1_FS3,D17,M2,FS3,7,0,138,6453,1645,0.635363,...,7.406103,6453.0,8.772455,36.246707,45.095305,55.726019,74.012087,41.0,3.737670,0.635363
1040_139,17D-M2-1_FS3,D17,M2,FS3,6,0,139,5708,1468,0.823406,...,7.292337,5708.0,8.649799,37.193413,46.303434,57.620883,76.822004,47.0,3.871201,0.823406
1040_140,17D-M2-1_FS3,D17,M2,FS3,5,0,140,6226,1586,0.594282,...,7.369601,6226.0,8.736650,35.448121,44.763893,56.424671,75.072278,37.0,3.637586,0.594282
1040_141,17D-M2-1_FS3,D17,M2,FS3,5,0,141,5500,1422,0.690909,...,7.260523,5500.0,8.612685,35.345455,44.400000,56.072727,76.472727,38.0,3.663562,0.690909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168_95,17D-M2-1_FS3,D17,M2,FS3,7,128,95,10858,2329,0.534168,...,7.753624,10858.0,9.292749,33.799963,42.696629,53.158961,69.497145,58.0,4.077538,0.534168
1168_96,17D-M2-1_FS3,D17,M2,FS3,8,128,96,10857,2387,0.571060,...,7.778211,10857.0,9.292658,32.614903,41.180805,51.690154,68.085106,62.0,4.143135,0.571060
1168_97,17D-M2-1_FS3,D17,M2,FS3,9,128,97,12143,2467,0.584699,...,7.811163,12143.0,9.404591,34.291361,43.243021,54.097011,70.073293,71.0,4.276666,0.584699
1168_98,17D-M2-1_FS3,D17,M2,FS3,9,128,98,10388,2434,0.587216,...,7.797702,10388.0,9.248503,33.423181,41.750096,51.540239,66.894494,61.0,4.127134,0.587216


In [88]:
X = adata.X

# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Convert sparse matrix to dense numpy array
X_train = X_train.toarray()
X_val = X_val.toarray()
X_test = X_test.toarray()

# Convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_val = torch.FloatTensor(X_val)
y_val = torch.LongTensor(y_val)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

# Create DataLoader for train, validation, and test sets
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64)

In [89]:
# Define Model Architecture
class ClassifierModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ClassifierModel, self).__init__()
        # Define layers of the neural network
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [124]:
# Cross-Validation Routine
def cross_validation(train_loader, val_loader, num_epochs, lr):
    model = ClassifierModel(input_size=X_train.shape[1], num_classes=len(np.unique(y_train)))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_losses = []
        val_predictions = []
        val_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                val_loss = criterion(outputs, targets)
                val_losses.append(val_loss.item())
                _, predicted = torch.max(outputs, 1)
                val_predictions.extend(predicted.cpu().numpy())
                val_targets.extend(targets.cpu().numpy())
        
        val_loss_avg = np.mean(val_losses)
        val_accuracy = accuracy_score(val_targets, val_predictions)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss_avg}, Validation Accuracy: {val_accuracy}')
    
    return model

In [125]:
# Train Model
best_model = cross_validation(train_loader, val_loader, num_epochs=10, lr=0.001)

Epoch 1/10, Validation Loss: 0.6779153900487083, Validation Accuracy: 0.75240653682561
Epoch 2/10, Validation Loss: 0.5876349862132754, Validation Accuracy: 0.77367360644728
Epoch 3/10, Validation Loss: 0.5987211891583034, Validation Accuracy: 0.7707633758674726
Epoch 4/10, Validation Loss: 0.7092478156089783, Validation Accuracy: 0.7647190508171032
Epoch 5/10, Validation Loss: 0.6896656700542995, Validation Accuracy: 0.7712111036489814
Epoch 6/10, Validation Loss: 0.7099632897547313, Validation Accuracy: 0.7727781508842624
Epoch 7/10, Validation Loss: 0.7468639510018485, Validation Accuracy: 0.7754645175733155
Epoch 8/10, Validation Loss: 0.7666107905762536, Validation Accuracy: 0.7718826953212446
Epoch 9/10, Validation Loss: 0.7917106262275151, Validation Accuracy: 0.7750167897918065
Epoch 10/10, Validation Loss: 0.8153051793575287, Validation Accuracy: 0.7712111036489814


In [126]:
# Evaluate on Hold-Out Test Set
best_model.eval()
test_predictions = []
test_targets = []
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = best_model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_predictions.extend(predicted.cpu().numpy())
        test_targets.extend(targets.cpu().numpy())

test_accuracy = accuracy_score(test_targets, test_predictions)
print(f'Test Accuracy: {test_accuracy}')

Test Accuracy: 0.7658964714311303
